In [1]:
import os
import sys
import math
import numpy as np

from collections import deque
from typing import Optional

sys.path.append(os.path.abspath('../../'))

from anguilla.ds.rbtree import RBTree
from anguilla.hypervolume.exact import calculate_3d

In [2]:
p_ref = np.array([1.1 , 1.1 , 1.1], dtype=float)
ps = np.array([
    [6.56039859404455e-2, 0.4474014917277, 0.891923776019316],
    [3.74945443950542e-2, 3.1364039802686e-2, 0.998804513479922],
    [0.271275894554688, 0.962356894778677, 1.66911984440026e-2],
    [0.237023460537611, 0.468951509833942, 0.850825693417425],
    [8.35813910785332e-2, 0.199763306732937, 0.97627289850149],
    [1.99072649788403e-2, 0.433909411793732, 0.900736544810901],
    [9.60698311356187e-2, 0.977187045721721, 0.18940978121319],
    [2.68052822856208e-2, 2.30651870780559e-2, 0.999374541394087],
    [0.356223209018184, 0.309633114503212, 0.881607826507812],
    [0.127964409429531, 0.73123479272024, 0.670015513129912],
    [0.695473366395562, 0.588939459338073, 0.411663831140169],
    [0.930735605917613, 0.11813654121718, 0.346085234453039],
    [0.774030940645471, 2.83363630460836e-2, 0.632513362272141],
    [0.882561783965009, 4.80931050853475e-3, 0.470171849451808],
    [4.92340623346446e-3, 0.493836329534438, 0.869540936185878],
    [0.305054163869799, 0.219367569077876, 0.926725324323535],
    [0.575227233936948, 0.395585597387712, 0.715978815661927],
    [0.914091673974525, 0.168988399705031, 0.368618138912863],
    [0.225088318852838, 0.796785147906617, 0.560775067911755],
    [0.306941172015014, 0.203530333828304, 0.929710987422322],
    [0.185344081015371, 0.590388202293731, 0.785550343533082],
    [0.177181921358634, 0.67105558509432, 0.719924279669315],
    [0.668494587335475, 0.22012845825454, 0.710393164782469],
    [0.768639363955671, 0.256541291890516, 0.585986427942633],
    [0.403457020846225, 0.744309886218013, 0.532189088208334],
    [0.659545359568811, 0.641205442223306, 0.39224418355721],
    [0.156141960251846, 8.36191498217669e-2, 0.984188765446851],
    [0.246039496399399, 0.954377757574506, 0.16919711007753],
    [3.02243260456876e-2, 0.43842801306405, 0.898257962656493],
    [0.243139979715573, 0.104253945099703, 0.96437236853565],
    [0.343877707314699, 0.539556201272222, 0.768522757034998],
    [0.715293885551218, 0.689330705208567, 0.114794756629825],
    [1.27610149409238e-2, 9.47996983636579e-2, 0.995414573777096],
    [0.30565381275615, 0.792827267212719, 0.527257689476066],
    [0.43864576057661, 3.10389339442242e-2, 0.8981238674636],
])
total_volume = 0.60496383631719475

In [3]:
point3d_dt = np.dtype([('x', float), ('y', float), ('z', float)], align=True)

In [4]:
class SweepItem:
    """Models an item of the sweep structure."""
    def __init__(self, val: np.array, idx: int) -> None:
        self.val = val
        self.idx = idx
        
    def __repr__(self):
        return "Item({}, {})".format(self.val, self.idx)
        
class Box3D:
    """Models and axis-parallel box in 3D."""
    def __init__(self, lower: np.array, upper: np.array):
        self.lower = lower
        self.upper = upper
        
    def __repr__(self):
        return '[{}, {}]'.format(self.lower, self.upper)
        
    def volume(self):
        return (self.upper['x'] - self.lower['x']) \
               * (self.upper['y'] - self.lower['y']) \
               * (self.upper['z'] - self.lower['z'])

In [21]:
def contributions_3d(ps: np.ndarray, ref_p: Optional[np.ndarray]) -> np.array:
    n = len(ps)

    if n == 0:
        return np.empty()
    
    if ref_p is None:
        zero = np.array([(0, 0, 0)], dtype=point3d_dt)[0]
        ref_p = zero

    # Sort the points by their z-coordinate in ascending order
    sorted_idx = np.argsort(ps[:, 2])
    sorted_ps = ps[sorted_idx]
    
    # "Cast" to allow access by "key"
    ps = sorted_ps.view(dtype=point3d_dt).reshape(-1)
    ref_p = ref_p.view(dtype=point3d_dt)[0]
    
    # Initialization
    front_xy = RBTree()
    
    # Add boundary sentinels to the sweeping structure
    sentinel_x = np.array([(ref_p['x'], float('-inf'), float('-inf'))], dtype=point3d_dt)[0]
    front_xy[ref_p['x']] = SweepItem(sentinel_x, n)
    sentinel_y = np.array([(float('-inf'), ref_p['y'], float('-inf'))], dtype=point3d_dt)[0]
    front_xy[float('-inf')] = SweepItem(sentinel_y, n)

    # Create box lists and contribution vector
    boxes = [deque([]) for _ in range(n+1)]
    contribution = np.zeros(n+1, dtype=float)
    contribution[n] = float('nan')
    
    # Process first point
    p0 = ps[0]
    upper0 = ref_p.copy()
    upper0['z'] = float('nan')
    boxes[0].appendleft(Box3D(p0.copy(), upper0))
    front_xy[p0['x']] = SweepItem(p0, 0)

    # Create pending points list
    pending = [SweepItem(p, i) for i, p in enumerate(ps[1:], 1)]

    # Process the rest of the points
    for p in pending:

        # find greatest q_x, such that q_x <= p_x
        node_q = front_xy.lower_bound_by_key(p.val['x'])        
        left = node_q.item

        if not (p.val['y'] < left.val['y']):  # p is dominated by q
            continue
    
        # (a) Find all points dominated by p, remove them
        # from the sweeping structure and add them to
        # the processing list 'dominated'.
        node_s = front_xy.succ(node_q)
        right = node_s.item
        dominated = []
        while True:
            if p.val['y'] > right.val['y']:
                break
            dominated.append(right)
            node_d = node_s
            node_s = front_xy.succ(node_s)
            right = node_s.item
            front_xy.remove(node_d)
        front_xy[p.val['x']] = p
  
        # (b) Process "left" region
        # Notice the symmetry with the paper's algorithm
        vol = 0.0
        while any(boxes[left.idx]):
            b = boxes[left.idx][-1]
            if p.val['x'] < b.lower['x']:
                # This box is dominated at this z-level
                # so it can completed and added to the
                # volume contribution of the left neighbour.
                b.upper['z'] = p.val['z']
                vol += b.volume()
                boxes[left.idx].pop()
            else:
                if p.val['x'] < b.upper['x']:
                    # Stop removing boxes.
                    b.upper['z'] = p.val['z']
                    vol += b.volume()
                    # Modify box to reflect the dominance
                    # of the left neighbour in this part 
                    # of the L region.
                    b.upper['x'] = p.val['x']
                    b.upper['z'] = float('nan')
                    b.lower['z'] = p.val['z']
                    break
        contribution[left.idx] += vol
            
        # (c) Process the dominated points
        right_x = right.val['x']
        for d in reversed(dominated):
            while any(boxes[d.idx]):
                b = boxes[d.idx].pop()
                b.upper['z'] = p.val['z']
                contribution[d.idx] += b.volume()
            # Create new box
            upper = d.val.copy()
            upper['x'] = right_x
            upper['z'] = float('nan')
            lower = p.val.copy()
            lower['x'] = d.val['x']
            b = Box3D(lower, upper)
            boxes[p.idx].appendleft(b)
            right_x = d.val['x']

        upper = left.val.copy()
        upper['x'] = right_x
        upper['z'] = float('nan')
        lower = p.val.copy()
        b = Box3D(lower, upper)
        boxes[p.idx].appendleft(b)
            
        # (d) Process "right" region
        vol = 0.0
        right_x = right.val['x']
        while any(boxes[right.idx]):
            b = boxes[right.idx][0]
            if b.upper['y'] >= p.val['y']:
                b.upper['z'] = p.val['z']
                vol += b.volume()
                right_x = b.upper['x']
                boxes[right.idx].popleft()
            else:
                break

        if right_x > right.val['x']:
            upper = p.val.copy()
            upper['x'] = right_x
            upper['z'] = float('nan')
            lower = right.val.copy()
            lower['z'] = p.val['z']
            b = Box3D(lower, upper)
            boxes[right.idx].appendleft(b)
        contribution[right.idx] += vol
        
    # The paper uses a 'z sentinel' to close the remaining boxes.
    # Here we do it as in Shark's approach.
    for p in front_xy:
        while any(boxes[p.idx]):
            b = boxes[p.idx].pop()
            assert math.isnan(b.upper['z'])
            b.upper['z'] = ref_p['z']
            contribution[p.idx] += b.volume()   
    
    for bl in boxes:
        assert len(bl) == 0
    
    reverse_idx = np.argsort(sorted_idx)
    return contribution[:-1][reverse_idx]

In [23]:
contributions_3d(ps, p_ref)

array([6.66900683e-05, 1.27052063e-05, 1.45153473e-02, 6.80707501e-04,
       4.86460918e-04, 1.59635724e-04, 9.35237153e-03, 7.38816827e-04,
       9.73449354e-04, 3.73033034e-03, 2.27674976e-03, 3.05265939e-03,
       5.86625018e-03, 8.33617977e-03, 3.61509047e-03, 6.90217889e-05,
       2.67173642e-03, 7.66193810e-04, 1.43917818e-03, 7.22979757e-05,
       9.12794195e-04, 1.37620025e-03, 3.30936198e-03, 1.90143581e-03,
       1.71245687e-03, 2.49798368e-03, 1.69423751e-04, 1.50575051e-03,
       1.13226411e-05, 5.48890368e-04, 1.28565415e-03, 2.62821322e-02,
       6.36034460e-04, 7.75883941e-04, 5.34042831e-03])

In [24]:
%timeit contributions_3d(ps, p_ref)

4.52 ms ± 74.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
# Implements the bruteforce computation
def contributions_3d_naive(ps: np.ndarray, ref_p: Optional[np.ndarray]) -> np.array:
    if len(ps) == 0:
        return np.empty()
    
    if ref_p is None:
        ref_p = np.zeros_like(ps[0])

    contribution = np.zeros(len(ps))

    vol = calculate_3d(ps, p_ref)
    for i in range(len(ps)):
        qs = np.delete(ps, i, 0)
        contribution[i] = vol - calculate_3d(qs, p_ref)
    return contribution

In [26]:
contributions_3d_naive(ps, p_ref)

array([6.66900683e-05, 1.27052063e-05, 1.45153473e-02, 6.80707501e-04,
       4.86460918e-04, 1.59635724e-04, 9.35237153e-03, 7.38816827e-04,
       9.73449354e-04, 3.73033034e-03, 2.27674976e-03, 3.05265939e-03,
       5.86625018e-03, 8.33617977e-03, 3.61509047e-03, 6.90217889e-05,
       2.67173642e-03, 7.66193810e-04, 1.43917818e-03, 7.22979757e-05,
       9.12794195e-04, 1.37620025e-03, 3.30936198e-03, 1.90143581e-03,
       1.71245687e-03, 2.49798368e-03, 1.69423751e-04, 1.50575051e-03,
       1.13226411e-05, 5.48890368e-04, 1.28565415e-03, 2.62821322e-02,
       6.36034460e-04, 7.75883941e-04, 5.34042831e-03])

In [27]:
%timeit contributions_3d_naive(ps, p_ref)

32 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
